In [1]:
import numpy as np
import math
import itertools
from functools import cache

In [2]:
n = 10
q = 2
x=np.random.randint(0,q,(n),dtype=np.ubyte)

$D_t(x)$ denotes the $t$ deletion ball of a sequence $x$ of size $n, assuming $t<n$.\
$D_t(x)$ is all of the subsequences of size $t$ of $x$.


In [3]:
def get_deletion_ball(x, t, N=-1):
    if N == -1:
        return np.array(list(set(itertools.combinations(x, x.size-t))))
    else:
        result = set()
        for c in itertools.combinations(x, x.size-t):
            result.add(c)
            if len(result) == N:
                break

        return np.array(list(result))



r=1
D_x_1 = get_deletion_ball(x,r)
print(f'For {x} size {n} vector of alphabet {q} we get a radius {r} deletion ball of size {D_x_1.shape[0]}')

In [5]:
def get_composition_vector(x, q):
    k = np.zeros(q,dtype=int)
    for i in x:
        k[i] += 1
    return k

def get_ordering(x, q=2):
    k = get_composition_vector(x, q)
    t = np.argsort(-k, kind='stable') # -k so it will be in descending order
    c = k[t]
    return t, c

def number_of_runs(x):
    runs = 1
    prev_x = x[0]
    for i in x[1:]:
        if i != prev_x:
            runs += 1
        prev_x = i

    return runs

@cache
def get_maximal_deletion_ball_size(n, t, q=2):
    if n < t or t < 0:
        return 0
    if q == 1:
        return 1

    size = 0
    for i in range(t+1):
        size += math.comb(n-t, i) * get_maximal_deletion_ball_size(t, t-i, q-1)
    return size

@cache
def get_maximal_insertion_ball_size(n, t, q=2):
    if n < 0 or t < 0:
        return 0
    size = 0
    for i in range(t+1):
        size += math.comb(n+t, i)*int(math.pow((q-1),i))
    return size

@cache
def get_maximal_number_of_common_subsequences(n, t, q=2):
    if n <= t or t <= 0:
        return 0
    return get_maximal_deletion_ball_size(n,t,q) - get_maximal_deletion_ball_size(n-1,t,q) + get_maximal_deletion_ball_size(n-2,t-1,q)

In [1]:
k_x = get_composition_vector(x, q)
t_x, l_x = get_ordering(x, q)
print(x)
print(k_x)
print(t_x)
print(l_x)
print(x, number_of_runs(x))
print(get_maximal_deletion_ball_size(n,1))
print(get_maximal_insertion_ball_size(n,1))
print(get_maximal_number_of_common_subsequences(n,1))
print(get_maximal_number_of_common_subsequences(n,2))
print(get_maximal_number_of_common_subsequences(7,3, 3))

NameError: name 'get_composition_vector' is not defined

In [7]:
def get_subsequence_reconstruction_threshold(n, t, order_comp, q=2):
    for i in range(q):
        w_i = order_comp[i]
        tau_i = get_maximal_number_of_common_subsequences(n-i-1,t-i,q)
        if w_i > tau_i:
            return i

In [8]:
def get_u_a_i(U, a, i):
    if i == 1:
        return U[:,U[0,:] == a][1:]
    results = []
    for s in U.T:
        if len(np.where(s == a)[0]) == 0: continue
        if np.where(s == a)[0][0] == i-1:
            results.append(s)

    if len(results) == 0:
        return None
    
    return np.stack(results).T[i:]
    

The following algorithm reconstructs a sequence $X \in F_q^n$ given $N_q^-(n,t)+1$ different subsequences of $X$ of size $t$\
The algorithm parameters:
* **n**             - The original sequence size to reconstruct.
* **subsequences**  - known subsequences of $X$, this is a ndarray of shape $(t, k)$ where $k$ is at least $N_q^-(n,t)+1$.
* **q**             - Size of the alphabet, the alphabets is assumed to be $[0,1,\dots, q-1]$.
* **verbose**       - control verbosity.




In [9]:
def reconstruct_x_from_subsequences(n, subsequences, q=2, verbose=False):
    t = n-subsequences.shape[0]
    reconstruction = np.array([], dtype=int)
    if verbose:
        print(subsequences.shape)
    while t >= 1:
        order_perm, order_comp = get_ordering(subsequences[0], q)
        j = get_subsequence_reconstruction_threshold(n, t, order_comp, q)
        reconstruction = np.concatenate((reconstruction, order_perm[:j+1]))
        if verbose:
            print(j, reconstruction)

        n = n-j-1
        t = t-j    
        N = get_maximal_number_of_common_subsequences(n,t,q)+1
        subsequences = get_u_a_i(subsequences, order_perm[j], 1)
        subsequences = subsequences[:,:N]
        if verbose:
            print(subsequences.shape)
        
    return np.concatenate((reconstruction, subsequences.T[0]))

Let us try and calculate the complexity for q=2

In [10]:
n = 20
t = 2
q = 20


x=np.random.randint(0,q,(n),dtype=np.ubyte)
N = get_maximal_number_of_common_subsequences(n,t,q)+1
print(N)
D_x = get_deletion_ball(x, t, N)

while D_x.shape[0] < N:
    print('1')
    x=np.random.randint(0,q,(n),dtype=np.ubyte)
    D_x = get_deletion_ball(x, t, N)

38


In [11]:
subsequences = D_x.T
# print(subsequences.shape)
# print(get_next_subsequnces(subsequences,0).shape)
# print(get_next_subsequnces(subsequences,0))
reconstructed_x = reconstruct_x_from_subsequences(n, subsequences, q)
print(f'Reconstructed {reconstructed_x}')
print(f'From {x}')
print(np.array_equal(x, reconstructed_x))
# print(subsequences[0])

Reconstructed [10  8  0 10  1  3  5  5 11 14  3  4  7 14  9 17  1  9 17  9]
From [10  8  0 10  1  3  5  5 11 14  3  4  7 14  9 17  1  9 17  9]
True


In [12]:
def get_insertion_ball(x, t, q=2, N=-1):
    result = set()
    n = x.shape[0]
    for indices in itertools.combinations_with_replacement(range(n+1), t):
        for letters in itertools.product(range(q), repeat=t):
            result.add(tuple(np.insert(x, indices, letters)))

            if len(result) == N:
                return np.array(list(result))

    return np.array(list(result))



In [13]:
def get_m_vector(U, t, a):
    m_vec = np.zeros((t+1,), dtype=np.uint)
    for i in range(t+1):
        u_a_i = get_u_a_i(U, a, i+1)
        if u_a_i is None:
            m_vec[i] = 0
        else:
            m_vec[i] = u_a_i.shape[1]

    return m_vec

In [14]:
@cache
def get_maximal_number_of_common_supersequences(n, t):
    if t <= 0:
        return 0
    result = 0
    for i in range(1, t+1):
        result += get_maximal_insertion_ball_size(n,t-i)
    return 2*result

In [15]:
def get_first_x(supersequences, t):
    N = supersequences.shape[1]
    candidates = []
    for i in range(2):
        m_v = get_m_vector(supersequences, t, i)
        if m_v.sum() == N:
            candidates.append((i, m_v))

    if len(candidates) == 1:
        return candidates[0]

    U_0_1 = 0
    U_1_0 = 0

    for s in supersequences.T:
        indices_0 = np.where(s == 0)[0]
        indices_1 = np.where(s == 1)[0]
        if len(indices_0) == 0 or len(indices_1) == 0: continue
        if indices_0[0] < indices_1[0]:
            U_0_1 += 1
        else:
            U_1_0 += 1

        # Since len(candidates)>1 then both 0 and 1 are candidates 
        # and because we added them by order candidates[0] represents 0
    if U_0_1 > U_1_0:
        return candidates[0] 
    
    return candidates[1]
    



In [16]:
def get_supersequence_resonstruction_threshold(n, t, m_v):
    for i in range(t+1):
        w_i = m_v[i]
        tau_i = get_maximal_number_of_common_supersequences(n-1,t-i)
        if w_i > tau_i:
            return i


In [17]:
def reconstruct_x_from_supersequences(n, supersequences, verbose=False):
    t = supersequences.shape[0]-n
    reconstruction = np.array([], dtype=int)
    if verbose:
        print(supersequences.shape)
    while t >= 1:
        x_1, m_x_1 = get_first_x(supersequences, t)
        reconstruction = np.append(reconstruction, x_1)
        if n == 1: 
            return reconstruction
        j = get_supersequence_resonstruction_threshold(n, t, m_x_1)
        if verbose:
            print(j, reconstruction)

        supersequences = get_u_a_i(supersequences, x_1, j+1)
        n = n-1
        t = t-j  
        N = get_maximal_number_of_common_supersequences(n,t)+1
        supersequences = supersequences[:,:N]
        if verbose:
            print(supersequences.shape)

    return np.concatenate((reconstruction, supersequences.T[0]))

In [22]:
n = 100
t = 4
q = 2
x=np.random.randint(0,q,(n),dtype=np.ubyte)
N = get_maximal_number_of_common_supersequences(n, t)+1
I_x = get_insertion_ball(x, t, q, N)
# print(N)
# print(I_x.shape)
print(x)

[0 0 1 1 0 0 1 1 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0
 1 1 0 1 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1
 1 1 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 0 0 0 0 0 1 0]
Reconstructed [0 0 1 1 0 0 1 1 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0
 1 1 0 1 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1
 1 1 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 0 0 0 0 0 1 0], 100
From [0 0 1 1 0 0 1 1 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0
 1 1 0 1 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1
 1 1 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 0 0 0 0 0 1 0], 100
True


In [24]:
reconstructed_x = reconstruct_x_from_supersequences(n, I_x.T)
print(f'Reconstructed {reconstructed_x}')
print(f'From {x}')
print(np.array_equal(x, reconstructed_x))

Reconstructed [0 0 1 1 0 0 1 1 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0
 1 1 0 1 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1
 1 1 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 0 0 0 0 0 1 0]
From [0 0 1 1 0 0 1 1 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0
 1 1 0 1 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1
 1 1 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 0 0 0 0 0 1 0]
True
